In [11]:
!pip install opencv-contrib-python

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 6.8 MB/s eta 0:00:00:00:0100:01m


In [4]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('2.png')

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# Apply thresholding to create a binary image
_, thresh = cv2.threshold(blur, 200, 255, cv2.THRESH_BINARY_INV)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(len(contours))
# Loop through the contours to find the cross
for contour in contours:
    # Approximate the contour to reduce the number of points
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    
print(len(approx),approx)
#     # Check if the approximated contour has 12 vertices (a heuristic for a cross shape)
#     if len(approx) == 12:
#         # Compute the moments of the contour
#         M = cv2.moments(contour)
        
#         # Calculate the centroid (center of mass)
#         if M["m00"] != 0:
#             cX = int(M["m10"] / M["m00"])
#             cY = int(M["m01"] / M["m00"])
#         else:
#             cX, cY = 0, 0
        
#         # Draw the contour and the centroid on the image
#         cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
#         cv2.circle(image, (cX, cY), 7, (255, 0, 0), -1)
#         cv2.putText(image, "center", (cX - 20, cY - 20),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
#         # Break the loop after finding the cross
#         break

# # Display the result
# cv2.imshow('Image with Cross Center', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

1
4 [[[   0    0]]

 [[   0  788]]

 [[1113  788]]

 [[1113    0]]]


In [14]:
import cv2

In [8]:
from PIL import Image
import numpy as np

# Open the PNG image
image = Image.open('2.png')

# Convert the image to grayscale
grayscale_image = image.convert('L')

# Convert the grayscale image to a 2D NumPy array
array_2d = np.array(grayscale_image)

# Print the shape of the 2D array
print(array_2d.shape)


(789, 1114)


In [9]:
789*1114

878946

In [13]:
a=array_2d
H,W=a.shape
s=np.zeros((H+1,W+1))
s1=np.zeros((W+1))
for y in range(H):
    for x in range(W):
        s1[x+1]=s1[x]+a[y][x]
        s[y+1][x+1]=s1[x+1]+s[y][x+1]

def rec_sum(x1,y1,x2,y2):
    return s[y2][x2]-s[y1-1][x2]-s[y2][x1-1]+s[y1-1][x1-1]


In [14]:
rs=rec_sum
##### define the cross window #######
c_size=15-1 
c_corner=5-1

sc=np.zeros(s.shape)
for y in range(1,H-c_size):
    for x in range(1,W-c_size):
        x2=x+c_size
        y2=y+c_size
        res=rs(x,y,x2,y2)
        res-=rs(x,y,x+c_corner,y+c_corner)
        res-=rs(x2-c_corner,y,x2,y+c_corner)
        res-=rs(x,y2-c_corner,x+c_corner,y2)
        res-=rs(x2-c_corner,y2-c_corner,x2,y2)
        sc[y][x]=res


In [22]:
flat_sc=sc.flatten()
print(np.quantile(sc,0.98))
print(np.quantile(flat_sc,0.001))

3122.0
0.0


In [23]:
thd=np.quantile(sc,0.98)
fsc=[(sc[y][x],x,y) for y in range(1,H) for x in range(1,W)]
fsc.sort()

In [27]:
vis=np.zeros(s.shape)
idx=len(fsc)-1
cnt=0
dx=[-1,0,0,1]
dy=[0,1,-1,0]
while cnt<9:
    while True:
        _,x,y=fsc[idx]
        if vis[y][x]==0:
            break
        idx-=1
    stack=[(x,y)]
    cnt+=1
    while len(stack)>0:
        x,y=stack.pop()
        vis[y][x]=cnt
        for i in range(4):
            xx=x+dx[i]
            yy=y+dy[i]
            if vis[yy][xx]==0 and sc[yy][xx]>thd:
                stack.append((xx,yy))

blks=[[] for i in range(9)]
for y in range(1,H):
    for x in range(1,W):
        if vis[y][x]>0:
            blks[int(vis[y][x])-1].append((x,y))
        

In [28]:
from statistics import mean

for blk in blks:
    xs=[i[0] for i in blk]
    ys=[i[1] for i in blk]
    print(mean(xs),mean(ys))

147.4503856041131 347.14498714652956
962.489701338826 347.15911431513905
544.489701338826 347.15911431513905
544.443025038324 669.0889115993868
962.5283307810107 669.0791220010209
962.4748858447489 109.02384576357179
544.4360672440142 109.06062149770759
147.38343558282207 669.0930470347648
147.42944785276075 109.09304703476482


In [ ]:
H,W=a.shape
s=np.zeros((H+1,W+1))
s1=np.zeros((W+1))
for y in range(H):
    for x in range(W):
        s1[x+1]=s1[x]+a[y][x]
        s[y+1][x+1]=s1[x+1]+s[y][x+1]

def rec_sum(x1,y1,x2,y2):
    return s[y2][x2]-s[y1-1][x2]-s[y2][x1-1]+s[y1-1][x1-1]

rs=rec_sum
##### define the cross window #######
c_size=15-1 
c_corner=5-1

sc=np.zeros(s.shape)
for y in range(1,H-c_size):
    for x in range(1,W-c_size):
        x2=x+c_size
        y2=y+c_size
        res=rs(x,y,x2,y2)
        res-=rs(x,y,x+c_corner,y+c_corner)
        res-=rs(x2-c_corner,y,x2,y+c_corner)
        res-=rs(x,y2-c_corner,x+c_corner,y2)
        res-=rs(x2-c_corner,y2-c_corner,x2,y2)
        sc[y][x]=res
thd=np.quantile(sc,0.98)
fsc=[(sc[y][x],x,y) for y in range(1,H) for x in range(1,W)]
fsc.sort()
vis=np.zeros(s.shape)
idx=len(fsc)-1
cnt=0
dx=[-1,0,0,1]
dy=[0,1,-1,0]
while cnt<9:
    while True:
        _,x,y=fsc[idx]
        if vis[y][x]==0:
            break
        idx-=1
    stack=[(x,y)]
    cnt+=1
    while len(stack)>0:
        x,y=stack.pop()
        vis[y][x]=cnt
        for i in range(4):
            xx=x+dx[i]
            yy=y+dy[i]
            if vis[yy][xx]==0 and sc[yy][xx]>thd:
                stack.append((xx,yy))

blks=[[] for i in range(9)]
for y in range(1,H):
    for x in range(1,W):
        if vis[y][x]>0:
            blks[int(vis[y][x])-1].append((x,y))
        